In [ ]:
from __future__ import print_function

import sys
import numpy as np
from time import time

sys.path.append('/home/xilinx')
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))

    print("Start of \"" + sys.argv[0] + "\"")

    ol = Overlay("./dataflow.bit")
    ipDesign = ol.example_0
    ipDMAIn0 = ol.axi_dma_0
    ipDMAIn1 = ol.axi_dma_1
    ipDMAOut0 = ol.axi_dma_2

    inBuffer0 = allocate(shape=(10,), dtype=np.int32)
    inBuffer1 = allocate(shape=(10,), dtype=np.int32)
    outBuffer0 = allocate(shape=(1,), dtype=np.int32)
    
 
    a=[0,1,2,3,4,5,6,7,8,9]
    b=[0,1,2,3,4,5,6,7,8,9]
    sum = 0
    for i in range(10):
        if b[i] != 0:
            sum += a[b[i]]
    print('A:')
    for i in range(10):
        print('{}'.format(a[i]), end=' ')
    print('')
    print('B:')
    for i in range(10):
        print('{}'.format(b[i]), end=' ')
    print('')
    print('software sum:')
    print('sum = {}'.format(c)) 
    
    for i in range(10):
        inBuffer0[i] = a[i]
        inBuffer1[i] = b[i]
    
    timeKernelStart = time()

    ipDesign.write(0x00, 0x01)
    ipDMAIn0.sendchannel.transfer(inBuffer0)
    ipDMAIn1.sendchannel.transfer(inBuffer1)
    ipDMAOut0.recvchannel.transfer(outBuffer0)
    ipDMAIn0.sendchannel.wait()
    ipDMAIn1.sendchannel.wait()
    ipDMAOut0.recvchannel.wait()
    
    timeKernelEnd = time()
    
    print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
    print('output = {}'.format(outBuffer0[0]))
    
    
    print("============================")
    print("Exit process")